In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
from gym import Env, spaces
from qpython import qconnection
import numpy as np
import json

In [9]:
def _exec(qry, pandas=False):
        with qconnection.QConnection(
            host="localhost", 
            port=5050, 
            pandas=pandas) as q:
            return q.sendSync(qry) 

In [17]:
data = _exec(".Q.ind[`events;30]", pandas=True)

In [20]:
data["cmd"]=(data["cmd"]).apply(lambda x:x.decode("utf-8"))
data["kind"]=(data["kind"]).apply(lambda x:x.decode("utf-8"))

In [22]:
q="""
derive:{[u]
    time:u[`resp][`data][`timestamp]; // should use this as ingress time
    a:flip u[`resp][`data][`asks][0];
    b:flip u[`resp][`data][`bids][0]; // should use utctime as egress time.
    :(20#"Z"$time;20#u[`utc_time];((10#`SELL),(10#`BUY));`int$((a[0],b[0])*100);`int$(a[1],b[1]));
};
x:derive each .Q.ind[orderbook;til 5000];
x:flip `time`intime`side`price`size!raze each flip x; 
x
"""
orderbook = _exec(q,pandas=True)

In [26]:
q="""
derive:{[u]
    time:u[`resp][`data][`timestamp]; // should use this as ingress time
    a:flip u[`resp][`data][`asks][0];
    b:flip u[`resp][`data][`bids][0]; // should use utctime as egress time.
    :(20#"Z"$time;20#u[`utc_time];((10#`SELL),(10#`BUY));`int$((a[0],b[0])*100);`int$(a[1],b[1]));
};
x:derive each .Q.ind[orderbookl;til 5000];
x:flip `time`intime`side`price`size!raze each flip x; 
x
"""
orderbookl = _exec(q,pandas=True)

QException: b'rank'

In [24]:
q:"""
derive:{d:x[`resp][`data];:("Z"$d[`timestamp]; count[d]#x[`utc_time]; upper `$d[`side]; `int$(d[`price]*100); `int$d[`size])};
x: derive each .Q.ind[trade;til 5000];
x:flip raze each flip x;
x
"""
trades = _exec(q,pandas=True)